
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

### Prepare and configure SparkSession with Delta Lake configuration

In [5]:
%session_id_prefix native-delta-sql-
%glue_version 3.0
%idle_timeout 60
%%configure 
{
  "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
  "--datalake-formats": "delta"
}

Setting session ID prefix to native-delta-sql-
Setting Glue version to: 3.0
Current idle_timeout is 15 minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog', '--datalake-formats': 'delta'}


In [1]:
bucket_name = "glue-deltalake-demo-us-east-1"

database_name = "deltalake_db"
database_location = f"s3://{bucket_name}/{database_name}/"

table_name = "products"
table_location = f"s3://{bucket_name}/{database_name}/{table_name}/"

print(f"database_location: {database_location}")
print(f"table_location: {table_location}")

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::123456789012:role/AWSGlueServiceRole-StudioNotebook
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: native-delta-sql--162db4ea-e072-44c3-ba6e-290bfddca0d4
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog
--datalake-formats delta
Waiting for session native-delta-sql--162db4ea-e072-44c3-ba6e-290bfddca0d4 to get into ready status...
Session native-delta-sql--162db4ea-e072-44c3-ba6e-290bfddca0d4 has been created.
database_location: s3://glue-deltalake-demo-us-east-1/deltalake_db/
table_location: s3://glue-deltalake-demo-us-east-1/deltalake_db/products/


### Create a Delta Lake table

In [2]:
%%sql
CREATE DATABASE IF NOT EXISTS deltalake_db

++
||
++
++


In [3]:
# create table in metastore
query = f"""
CREATE TABLE IF NOT EXISTS {database_name}.{table_name} (
  product_id STRING,
  product_name STRING,
  price INT,
  category STRING,
  updated_at TIMESTAMP
)
USING delta
PARTITIONED BY (category)
LOCATION '{table_location}'
"""

spark.sql(query)

DataFrame[]


In [4]:
%%sql
USE deltalake_db

++
||
++
++


In [5]:
%%sql
SHOW TABLES

+------------+---------+-----------+
|    database|tableName|isTemporary|
+------------+---------+-----------+
|deltalake_db| products|      false|
+------------+---------+-----------+


### Read from Delta Lake table

In [6]:
%%sql # Read table from metastore
SELECT * FROM deltalake_db.products

+----------+--------------------+-----+-------------+-------------------+
|product_id|        product_name|price|     category|         updated_at|
+----------+--------------------+-----+-------------+-------------------+
|     00007|Mercedes-Benz C-C...| 7341|       Nissan|2023-02-15 01:26:27|
|     00019|   Cadillac De Ville| 5707|       Nissan|2023-02-15 12:17:03|
|     00015|    Chevrolet Impala| 3128|      Maybach|2023-02-15 00:41:04|
|     00004|    Chevrolet Impala| 4054|Mercedes-Benz|2023-02-15 06:05:01|
|     00012|     ZAZ Zaporozhets| 5665|         Fiat|2023-02-15 11:11:35|
|     00003|      Imperial Crown| 4803|        Mazda|2023-02-15 12:36:15|
|     00010|       Ford F-Series| 5070|        Mazda|2023-02-15 05:10:21|
|     00011|        Metropolitan| 3622|      Maybach|2023-02-15 06:09:05|
|     00013|         Alpine A310| 4923|    Chevrolet|2023-02-15 01:30:13|
|     00006|            Fiat Uno| 6399|       Lancia|2023-02-15 02:22:30|
|     00016|      Koenigsegg CCX| 2261

### View History

In [7]:
query = f"""DESCRIBE HISTORY delta.`{table_location}` """
spark.sql(query).show(truncate=False)

+-------+-------------------+------+--------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|version|timestamp          |userId|userName|operation   |operationParameters                                                                                                                                    |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                 

### Query with time travel

In [8]:
query = f"""SELECT * FROM delta.`{table_location}@v1`""" # Using a version number
spark.sql(query).show(truncate=False)

+----------+-----------------+-----+----------+-------------------+
|product_id|product_name     |price|category  |updated_at         |
+----------+-----------------+-----+----------+-------------------+
|00019     |Cadillac De Ville|5707 |Nissan    |2023-02-15 12:17:03|
|00009     |De Tomaso Pantera|7232 |Chevrolet |2023-02-15 04:20:14|
|00020     |Volkswagen Beetle|2018 |Volkswagen|2023-02-15 03:18:10|
|00016     |Metropolitan     |8818 |Mazda     |2023-02-15 03:10:14|
|00013     |Ford Model T     |2404 |Maybach   |2023-02-15 05:16:19|
|00015     |Chevrolet Impala |3128 |Maybach   |2023-02-15 00:41:04|
|00008     |Peugeot 206      |2855 |Mazda     |2023-02-15 06:28:07|
|00014     |Toyota Corolla   |7866 |Mazda     |2023-02-15 00:04:17|
+----------+-----------------+-----+----------+-------------------+


In [ ]:
%stop_session

Stopping session: native-delta-sql--162db4ea-e072-44c3-ba6e-290bfddca0d4
Stopped session.
